In [1]:
import mne 
import os
import mne 
from mne.io import read_raw
import numpy as np
import pandas as pd

In [2]:
data_folder = "data/"
freq = 80
q = 256/freq

# Set the path to the patient folder
data_files = sorted([f for f in os.listdir(data_folder) if f.startswith('chb')])
# Define the list of channels to keep
# channels_to_keep = ['FP1-F7', 'FP1-F3', 'FP2-F4', 'F8-T8','FP2-F8', 'T8-P8-1', 'T8-P8-0', 'P8-O2', 'P4-O2']# 
channels_to_keep = ['T7-P7', 'T8-P8-0']# 
dataframes = []  # List to store individual DataFrames

for folder_list in range(1,len(data_files[:5])):  # Assumes folders are named ch01 to ch10
        folder_name = f'chb{folder_list:02d}'  # Format the folder name as ch01, ch02, etc.
        folder_path = os.path.join(data_folder,folder_name)  # Create the full path to the folder

        # Create an empty list to store raw data
        patient_raws = []

        # Create an empty list to store file names
        file_names = []

        # Get a list of .edf files sorted by filename numbers
        edf_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.edf')])

        # Read the first .edf file to obtain the channel information
        first_file_path = os.path.join(folder_path, edf_files[0])
        first_raw = mne.io.read_raw_edf(first_file_path, preload=False, verbose="ERROR")
        first_raw.pick_channels(channels_to_keep)

        # Loop through the sorted .edf files
        for file in edf_files:
            file_path = os.path.join(folder_path, file)
            raw = mne.io.read_raw_edf(file_path, preload=False, verbose="ERROR")
            #Keep only the specified channels
            raw.pick_channels(channels_to_keep)
            raw_time_df = raw.to_data_frame()
            new_time = raw_time_df["time"].iloc[::2].reset_index()
            #raw_time_df = raw_time_df["time"]
            raw2 = raw.resample(sfreq=freq)

            # Set the channel information for each Raw object to match the first Raw object
            raw2.info = first_raw.info
            raw_df = raw2.to_data_frame()
            raw_df["file_name"] = file
            raw_df["original_time"] = new_time["time"]
            patient_raws.append(raw_df)
            #file_names.extend([file] * len(raw.times))

        # Concatenate the raw objects into a single patient_combined object while maintaining a continuous time axis
        #patient_combined = mne.concatenate_raws(patient_raws, preload=False)
        patient_combined = pd.concat(patient_raws, ignore_index=True)
        # Print info about the combined raw object for this patient
        #print(patient_combined.info)

        #df = patient_combined.to_data_frame()
        #df['file_name'] = file_names
        dataframes.append(patient_combined)
# Concatenate all DataFrames into a single DataFrame
final_dataframe = pd.concat(dataframes, ignore_index=True)

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy functi

In [6]:
final_dataframe.head(100)

,time,T7-P7,T8-P8-0,file_name,original_time
0,0.000000,-42.789594,44.710026,chb01_01.edf,0.000000
1,0.003906,8.051760,-7.175426,chb01_01.edf,0.007812
2,0.007812,-4.458138,4.714653,chb01_01.edf,0.015625
3,0.011719,6.754814,-1.980045,chb01_01.edf,0.023438
4,0.015625,14.303701,10.122724,chb01_01.edf,0.031250
...,...,...,...,...,...
95,0.371094,14.630467,-13.569140,chb01_01.edf,0.742188
96,0.375000,25.085259,-0.441500,chb01_01.edf,0.750000
97,0.378906,11.949023,5.744442,chb01_01.edf,0.757812
98,0.382812,1.804679,-9.680896,chb01_01.edf,0.765625


In [7]:
final_dataframe.to_csv('final_dataframe.csv')

In [8]:
final_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46406240 entries, 0 to 46406239
Data columns (total 5 columns):
 #   Column         Dtype  
---  ------         -----  
 0   time           float64
 1   T7-P7          float64
 2   T8-P8-0        float64
 3   file_name      object 
 4   original_time  float64
dtypes: float64(4), object(1)
memory usage: 1.7+ GB


In [59]:
def get_seizure_sequence(source_path="data"):
    all_data = []

    for folder_name in os.listdir(source_path):
        folder_path = os.path.join(source_path, folder_name)

        if os.path.isdir(folder_path):
            for filename in os.listdir(folder_path):
                if filename.endswith(".txt"):
                    file_path = os.path.join(folder_path, filename)
                    source_file = filename.split("-")[0]
                    data = []
                    current_data = {"File Name": None, "Number of Seizures": None, "Seizure Start Time": np.nan, "Seizure End Time": np.nan, "Source File": source_file}
                    with open(file_path, "r") as file:
                        for line in file:
                            line = line.strip()
                            if line.startswith("File Name:"):
                                #current_data = {"File Name": None, "Number of Seizures": None, "Seizure Start Time": np.nan, "Seizure End Time": np.nan, "Source File": source_file}
                                current_data["File Name"] = line.split(": ", 1)[1]
                            elif line.startswith("Number of Seizures in File:"):
                                current_data["Number of Seizures"] = int(line.split(": ", 1)[1])
                                if current_data["Number of Seizures"] == 0:
                                    data.append(current_data.copy())
                            elif line.startswith("Seizure Start Time:"):
                                current_data["Seizure Start Time"] = float(line.split(": ", 1)[1].split()[0])
                            elif line.startswith("Seizure End Time:"):
                                current_data["Seizure End Time"] = float(line.split(": ", 1)[1].split()[0])
                                data.append(current_data.copy())
                    all_data.extend(data)

    seizure_seq = pd.DataFrame(all_data)
    seizure_seq.columns = seizure_seq.columns.str.lower().str.replace(" ", "_")
    return seizure_seq

In [60]:
seizure_sequences = get_seizure_sequence("data")
seizure_sequences["file_name"].unique()

array(['chb11_01.edf', 'chb11_02.edf', 'chb11_03.edf', 'chb11_04.edf',
       'chb11_05.edf', 'chb11_06.edf', 'chb11_07.edf', 'chb11_08.edf',
       'chb11_09.edf', 'chb11_10.edf', 'chb11_11.edf', 'chb11_12.edf',
       'chb11_13.edf', 'chb11_14.edf', 'chb11_15.edf', 'chb11_16.edf',
       'chb11_17.edf', 'chb11_18.edf', 'chb11_19.edf', 'chb11_24.edf',
       'chb11_25.edf', 'chb11_26.edf', 'chb11_27.edf', 'chb11_53.edf',
       'chb11_54.edf', 'chb11_55.edf', 'chb11_56.edf', 'chb11_58.edf',
       'chb11_60.edf', 'chb11_61.edf', 'chb11_62.edf', 'chb11_63.edf',
       'chb16_01.edf', 'chb16_02.edf', 'chb16_03.edf', 'chb16_04.edf',
       'chb16_05.edf', 'chb16_06.edf', 'chb16_07.edf', 'chb16_08.edf',
       'chb16_09.edf', 'chb16_12.edf', 'chb16_13.edf', 'chb16_15.edf',
       'chb16_19.edf', 'chb20_01.edf', 'chb20_02.edf', 'chb20_03.edf',
       'chb20_04.edf', 'chb20_05.edf', 'chb20_06.edf', 'chb20_07.edf',
       'chb20_08.edf', 'chb20_11.edf', 'chb20_17.edf', 'chb20_21.edf',
      

In [61]:
x = seizure_sequences[seizure_sequences["number_of_seizures"] == 2]
x["file_name"].unique()


array([], dtype=object)

In [ ]:
a = df

In [43]:
# Add the number of seizures per dataframe
merged_df = final_dataframe.merge(x, on='file_name', how='left')
# Fill NaN values in the new column with 0 (no seizures by default)
merged_df['number_of_seizures'].fillna(0, inplace=True)
merged_df

,time,FP1-F7,FP1-F3,file_name,original_time,number_of_seizures
0,0.000000,-145.889824,-170.898152,chb01_01.edf,0.000000,0.0
1,0.003906,19.998681,23.455757,chb01_01.edf,0.007812,0.0
2,0.007812,-10.400608,-12.495403,chb01_01.edf,0.015625,0.0
3,0.011719,7.521448,7.968614,chb01_01.edf,0.023438,0.0
4,0.015625,-6.548636,-6.684477,chb01_01.edf,0.031250,0.0
...,...,...,...,...,...,...
120676987,7199.980469,24.672520,16.816139,chb04_43.edf,14399.960938,0.0
120676988,7199.984375,24.421689,23.013589,chb04_43.edf,14399.968750,0.0
120676989,7199.988281,21.769350,24.298143,chb04_43.edf,14399.976562,0.0
120676990,7199.992188,5.203675,2.576982,chb04_43.edf,14399.984375,0.0


In [44]:
merged_df["number_of_seizures"].unique()

array([0., 1.])

In [9]:
# This function adds if a timestamp is in a seizure sequence
def assign_seizures(final_df, seizures_df):
    final_df["Seizure"] = 0
    
    # Iterate through each row in participants_df and check if 'original_time' is within any seizure range
    for index, row in final_df.iterrows():
        file_name = row["file_name"]
        original_time = row["original_time"]
        matching_seizure = seizures_df[(seizures_df["file_name"] == file_name) & (seizures_df["Seizure Start Time"] <= original_time) & (seizures_df["Seizure End Time"] >= original_time)]
        
        if not matching_seizure.empty:
            final_df.at[index, "Seizure"] = 1
    
    return final_df